# CIRCexplorer - conda environment (oct_env)
https://circexplorer2.readthedocs.io/en/latest/tutorial/setup/#installation-and-setup

## Get reference genome/files

In [ ]:
fetch_ucsc.py mm10 ref mm10_ref.txt
fetch_ucsc.py mm10 kg mm10_kg.txt
fetch_ucsc.py mm10 fa mm10.fa
cut -f2-11 mm10_ref.txt|./data/reference/genePredToGtf file stdin mm10_ref.gtf
cut -f2-11 mm10_kg.txt|./data/reference/genePredToGtf file stdin mm10_kg.gtf

## create Bowtie2 index files

In [ ]:
bowtie2-build mm10.fa bowtie2_index
# this creates the files bowtie2_index.<1-4>.bt2 and bowtie2_index.rev.<1-2>.bt2
cp mm10.fa bowtie2_index.fa

## download Boost.1.70.0
https://www.boost.org/users/download/

In [ ]:
./bootstrap
./bjam --prefix=/usr/local/bin/ link=static runtime-link=static stage install

## download Tophat-2.1.1 binary
http://ccb.jhu.edu/software/tophat/downloads/
change Shebang in tophat file from #/bin/usr/python to #/bin/usr/python2
https://ccb.jhu.edu/software/tophat/tutorial.shtml

In [ ]:
conda install -c bioconda tophat
conda install -c bioconda bowtie
conda install -c bioconda bowtie2

In [ ]:
CIRCexplorer2 align -G data/reference/mm10_kg.gtf -g data/reference/mm10.fa -f data/mouse_brain_GSE100265/fastq/rRNA
depleted_SRP109830/SRR5720118_1.fastq > CIRCexplorer2_align_SRR5720118_1.log


#### Error: cannot map unmapped reads with TopHat-Fusion

## Run tophat2

In [ ]:
# first trial - just a test
bash circRNA-detection/soft/tophat-2.1.1.Linux_x86_64/tophat2 -a 6 --microexon-search -m 2 -p 10 -G data/reference/mm10_kg.gtf -o tophat data/reference/bowtie2_index data/mouse_brain_GSE100265/fastq/rRNAdepleted_SRP109830/SRR5720118_1.fastq
# second trial
bash circRNA-detection/soft/tophat-2.1.1.Linux_x86_64/tophat2 -o tophat_output_SRR5720118 data/reference/bowtie2_index data/mouse_brain_GSE100265/fastq/rRNAdepleted_SRP109830/SRR5720118_1.fastq

## Test TopHat and Bowtie installation with test_data
#Works fine
test_ref.1.bt2
test_ref.2.bt2
test_ref.3.bt2
test_ref.4.bt2
test_ref.rev.1.bt2
test_ref.rev.2.bt2

test_ref.1.ebwt
test_ref.2.ebwt
test_ref.rev.1.ebwt
test_ref.rev.2.ebwt

test_ref.fa

In [ ]:
/data/home/students/ciora/test_data$ tophat -r 20 test_ref reads_1.fq reads_2.fq

## Manually aligning one Fastq file

Testing
~/testing
bowtie1_index
bowtie2_index
mm10.fa
mm10_kg.gtf
SRR5720118_1.fastq

In [ ]:
# 3759.pts-1.alfred
# index genome for Bowtie
bowtie-build mm10.fa bowtie1_index/mm10


In [ ]:
# 9695.pts-7.alfred
# index genome for Bowtie2
bowtie2-build mm10.fa bowtie2_index/mm10

In [ ]:
tophat2 -a 6 --microexon-search -m 2 -p 10 -G mm10_kg.gtf -o tophat_out bowtie2_index/mm10 SRR5720118_1.fastq
# done ~ 2 hours

Input: 19900102
Mapped: 17130031 (86.1% of input)
of these: 624706 ( 3.6%) have multiple alignments (1166 have >20)
86.1% overall read mapping rate.

In [ ]:
bamToFastq -i tophat_out/unmapped.bam -fq tophat_out/unmapped.fastq

In [ ]:
tophat2 -o tophat_fusion -p 15 --fusion-search --keep-fasta-order --bowtie1 --no-coverage-search bowtie1_index/mm10 tophat_out/unmapped.fastq

In [ ]:
[2019-06-23 21:05:06] Beginning TopHat run (v2.1.1)
-----------------------------------------------
[2019-06-23 21:05:06] Checking for Bowtie
                  Bowtie version:        1.2.2.0
[2019-06-23 21:05:06] Checking for Bowtie index files (genome)..
[2019-06-23 21:05:06] Checking for reference FASTA file
[2019-06-23 21:05:06] Generating SAM header for bowtie1_index/mm10
[2019-06-23 21:05:08] Preparing reads
         left reads: min. length=35, max. length=151, 2752055 kept reads (18016 discarded)
[2019-06-23 21:06:10] Mapping left_kept_reads to genome mm10 with Bowtie
        [FAILED]
Error running bowtie:
Error while flushing and closing output
terminate called after throwing an instance of 'int'

last step with bowtie2 instead of bowtie1

In [ ]:
tophat2 -o tophat_fusion -p 15 --fusion-search --keep-fasta-order --no-coverage-search bowtie2_ind
ex/mm10 tophat_out/unmapped.fastq

In [ ]:
[2019-06-23 23:04:20] Reporting output tracks
        [FAILED]
Error running /data/home/students/ciora/.conda/envs/oct_env/bin/tophat_reports --min-anchor 8 --splice-mismatches 0 --min-report-intron 50 --max-report-intron 500000 --min-isoform-fraction 0.15 --output-dir tophat_fusion/ --max-multihits 20 --max-seg-multihits 40 --segment-length 25 --segment-mismatches 2 --min-closure-exon 100 --min-closure-intron 50 --max-closure-intron 5000 --min-coverage-intron 50 --max-coverage-intron 20000 --min-segment-intron 50 --max-segment-intron 500000 --read-mismatches 2 --read-gap-length 2 --read-edit-dist 2 --read-realign-edit-dist 3 --max-insertion-length 3 --max-deletion-length 3 --fusion-search --fusion-anchor-length 20 --fusion-min-dist 10000000 --fusion-read-mismatches 2 --fusion-multireads 2 --fusion-multipairs 2 -z gzip -p15 --no-closure-search --no-coverage-search --no-microexon-search --sam-header tophat_fusion/tmp/mm10_genome.bwt.samheader.sam --report-discordant-pair-alignments --report-mixed-alignments --samtools=/data/home/students/ciora/.conda/envs/oct_env/bin/samtools_0.1.18 --bowtie2-max-penalty 6 --bowtie2-min-penalty 2 --bowtie2-penalty-for-N 1 --bowtie2-read-gap-open 5 --bowtie2-read-gap-cont 3 --bowtie2-ref-gap-open 5 --bowtie2-ref-gap-cont 3 bowtie2_index/mm10.fa tophat_fusion/junctions.bed tophat_fusion/insertions.bed tophat_fusion/deletions.bed tophat_fusion/fusions.out tophat_fusion/tmp/accepted_hits tophat_fusion/tmp/left_kept_reads.mapped.bam,tophat_fusion/tmp/left_kept_reads.candidates tophat_fusion/tmp/left_kept_reads.bam
./SeqAn-1.4.2/seqan/basic/basic_exception.h:236 FAILED!  (Uncaught exception of type St12out_of_range: basic_string::substr)

In [ ]:
conda install -c bioconda tophat=2.1.0
conda install -c bioconda bowtie2=2.2.1

In [ ]:
tophat2 -o tophat_fusion -p 15 --fusion-search --keep-fasta-order --no-coverage-search bowtie2_ind
ex/mm10 tophat_out/unmapped.fastq

In [ ]:
Warning: --fusion-search with Bowtie2 may not work well as it may require much memory space and produce many spurious fusions.  Please try --bowtie1 option if this doesn't work.

[2019-06-23 23:15:13] Beginning TopHat run (v2.1.0)
-----------------------------------------------
[2019-06-23 23:15:13] Checking for Bowtie
                  Bowtie version:        2.2.1.0

In [ ]:
Reads:
Input:   2770071
Mapped:    469621 (17.0% of input)
of these:     42610 ( 9.1%) have multiple alignments (219 have >20)
17.0% overall read mapping rate.

In [ ]:
CIRCexplorer2 parse -t TopHat-Fusion tophat_fusion/accepted_hits.bam > CIRCexplorer2_parse.log

In [ ]:
CIRCexplorer2 annotate -r mm10_ref.txt -g mm10.fa -b back_spliced_junction.bed -o circularRNA_known.txt > CIRCexplorer2_annotate.log

### This worked!

# Circtools - venv environment (py2_env)
https://docs.circ.tools/en/latest/Detect.html
virtualenv --python=/usr/bin/python2.7 py2_env
source py2_env/bin/activate

In [ ]:
git clone https://github.com/dieterich-lab/circtools.git
cd circtools
pip install . install --verbose --user

(py2_env) (oct_env) pip3 install circtools --user
installed circtools, installed DCC

### install STAR
conda install -c bioconda star
STAR_2.6.1a_08-27

### build STAR index files

In [ ]:
# 37461.pts-1.alfred
STAR --runThreadN 16 --runMode genomeGenerate --genomeDir circtools_method/star/star_index_mm10/ --genomeFastaFiles data/reference/mm10.fa --sjdbGTFfile data/reference/mm10_ref.gtf

In [ ]:
Jun 24 14:37:51 ..... started STAR run
Jun 24 14:37:51 ... starting to generate Genome files
Jun 24 14:39:09 ... starting to sort Suffix Array. This may take a long time...
Jun 24 14:39:45 ... sorting Suffix Array chunks and saving them to disk...
Jun 24 14:55:47 ... loading chunks from disk, packing SA...
Jun 24 14:57:21 ... finished generating suffix array
Jun 24 14:57:21 ... generating Suffix Array index
Jun 24 15:03:06 ... completed Suffix Array index
Jun 24 15:03:06 ..... processing annotations GTF
Jun 24 15:03:10 ..... inserting junctions into the genome indices
Jun 24 15:06:04 ... writing Genome to disk ...
Jun 24 15:06:09 ... writing Suffix Array to disk ...
Jun 24 15:06:57 ... writing SAindex to disk
Jun 24 15:07:01 ..... finished successfully

### Mapping with STAR

In [ ]:
STAR --runThreadN 16 --genomeDir ~/circtools_method/star/star_index_mm10/ --readFilesIn data/sample1/SRR5720118_1.fastq --outFileNamePrefix SRR5720118_1 --chimOutType Junctions
SeparateSAMold --sjdbGTFfile ~/data/reference/mm10_ref.gtf

# Keep processes running

In [ ]:
screen # start process
ctrl+A ctrl+D # deattach
screen -r # resume